![Practicum AI Logo image](images/practicum_ai_logo.png)
***
This exercise adapted from Baig et al. (2020) <i>The Deep Learning Workshop</i> from <a href="https://www.packtpub.com/product/the-deep-learning-workshop/9781839219856">Packt Publishers</a>.

#### Exercise 2.01 (Student) - page 55

***

![](images/02.1_image_1.jpg)

<span style="color:steelblue">**(1)** Import the Packages</span>

In [1]:
# Code it!


![](images/02.1_image_2.jpg)

<span style="color:steelblue">**(2)** Set Input Data</span>

In [4]:
# Code it!


![](images/02.1_image_3.jpg)

<span style="color:steelblue">**(3)** Set Actual Labels</span>

In [5]:
# Code it!


<span style="color:steelblue">**(4)** Define Perceptron Parameters</span>

In [6]:
# Code it!


![](images/02.1_image_5.jpg)

<span style="color:steelblue">**(5)** Define the Connections Weight Matrix</span>

In [7]:
# Code it!


![](images/02.1_image_6.jpg)

<span style="color:steelblue">**(6)** Create the Variable for the Bias</span>

In [8]:
# Code it!


![](images/02.1_image_7.jpg)

<span style="color:steelblue">**(7)** Define the Perceptron</span>

In [25]:
def perceptron(X):
    z = tf.add(tf.matmul(X, W), B)
    output = activations.sigmoid(z)
    return output

In [24]:
print(perceptron(X))

tf.Tensor(
[[0.43065366]
 [0.72677964]
 [0.72677964]
 [0.90342504]], shape=(4, 1), dtype=float32)


### Back Propogation - page 60

<div style="padding: 10px;margin-bottom: 20px;border: thin solid #6377B2;border-left-width: 25px;background-color: #fff"><strong>Note:</strong> The section (Training a Perceptron) provides code blocks to complete this part of the worksheet, though they are not numbered. </div>

In [3]:
# Code it! -- Assign a value to the learning_rate and optimizer variables (page 60)


In [ ]:
# Code it! -- Train the Perceptron for 2000 epochs


Click on the '...' below to show the solution.

In [12]:
no_of_epochs = 2000

for n in range(no_of_epochs):
    loss = lambda:abs(tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = y, logits = perceptron(X))))
    optimizer.minimize(loss, [W, B])

In [1]:
# Code it! -- Print the trained weights


In [2]:
# Code it! -- Print the bias


In [ ]:
print(perceptron(X))